# Cuaderno para realizar la exploracion en la busqueda de la homogeneizacion de las series

Partimos de utilizar los datos proporcionados por la pagina del [National Weather Service](http://www.nws.noaa.gov/) de los estados unidos, donde descargamos los datos del ONI(Oceanic Nino Index), de esta forma obtenemos los periodos donde han ocurrido los fenomenos del niño y de la niña.

[Cold & Warm Episodes by Season](https://www.cpc.ncep.noaa.gov/products/analysis_monitoring/ensostuff/ONI_v5.php)

In [32]:
import pandas as pd

columns = ['Year', 'January', 'February', 'March', 'April', 'May', 'June', 'July', 'August', 'September', 'October', 'November', 'December']
oni = pd.read_csv("../RecursosYDocumentos/ONI/ONI.csv", header=None, delimiter="\t", names=columns)
print(oni.head(5))
oni = oni.melt(id_vars=["Year"], var_name="month", value_name="oni")
oni["month_number"] = pd.to_datetime(oni["month"], format='%B').dt.month
oni = oni.sort_values(by=["Year", "month_number"])
oni.reset_index(drop=True, inplace=True)

oni["nino"] = oni["oni"] > 0.5
oni["nina"] = oni["oni"] < -0.5

oni = oni.astype({"nino": int, "nina": int})

print(oni.head(5))

   Year  January  February  March  April  May  June  July  August  September  \
0  1950     -1.5      -1.3   -1.2   -1.2 -1.1  -0.9  -0.5    -0.4       -0.4   
1  1951     -0.8      -0.5   -0.2    0.2  0.4   0.6   0.7     0.9        1.0   
2  1952      0.5       0.4    0.3    0.3  0.2   0.0  -0.1     0.0        0.2   
3  1953      0.4       0.6    0.6    0.7  0.8   0.8   0.7     0.7        0.8   
4  1954      0.8       0.5    0.0   -0.4 -0.5  -0.5  -0.6    -0.8       -0.9   

   October  November  December  
0     -0.4      -0.6      -0.8  
1      1.2       1.0       0.8  
2      0.1       0.0       0.1  
3      0.8       0.8       0.8  
4     -0.8      -0.7      -0.7  
   Year     month  oni  month_number  nino  nina
0  1950   January -1.5             1     0     1
1  1950  February -1.3             2     0     1
2  1950     March -1.2             3     0     1
3  1950     April -1.2             4     0     1
4  1950       May -1.1             5     0     1
